# 05 – Transformer Fine‑Tune (DistilBERT)

Fine‑tune a lightweight transformer (DistilBERT) on the **Twitter‑Airline Sentiment** dataset and benchmark it against a classical **TF‑IDF + Logistic Regression** baseline.

> **Model** `distilbert‑base‑uncased`  
> **Training split** 90 % of cleaned data (stratified)  
> **Validation split** 10 % (held‑out during fine‑tuning)  
> **Test set** Untouched split created in `04_baseline_model.ipynb`  
> **Artifacts saved to** `models/distilbert_twitter/`

## 0 Imports & Global Config

Everything we need in one place:

1. **Path handling** (`pathlib.Path`) so the notebook is platform‑agnostic.  
2. **Reproducibility seeds** for Python, NumPy, and (if available) CUDA.  
3. **Key Hugging Face classes** (`AutoTokenizer`, `AutoModelForSequenceClassification`, `Trainer`, …).  
4. A line that tells Transformers to **ignore TensorFlow** so only PyTorch is used.

In [ ]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

# --- core -------------------------------------------------------------
from pathlib import Path
import random

# --- third‑party ------------------------------------------------------
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# transformers / HF
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, DataCollatorWithPadding
)
from datasets import Dataset 
from evaluate import load as load_metric
import torch

# --- reproducibility --------------------------------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# --- paths ------------------------------------------------------------
PROJ_ROOT  = Path.cwd()
DATA_DIR   = PROJ_ROOT / "data" / "processed"
MODEL_DIR  = PROJ_ROOT / "models" / "distilbert_twitter"
MODEL_DIR.mkdir(parents=True, exist_ok=True)

c:\Users\justi\Anaconda3\envs\twitter-sentiment-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1 Load Cleaned Data

Read the parquet file that contains **14 640 pre‑cleaned tweets** and show the first few rows to confirm the schema.

In [3]:
PROJ_ROOT = Path.cwd().parent           # one level up from notebooks/
DATA_DIR  = PROJ_ROOT / "data" / "processed"

# Adjust filename to match actual parquet/CSV
df = pd.read_parquet(DATA_DIR / "tweets.parquet")

display(df.head())
print(f"Loaded {len(df):,} tweets")

,tweet_id,airline,airline_sentiment,clean_text,negativereason
0,570306133677760513,Virgin America,neutral,what said.,None
1,570301130888122368,Virgin America,positive,plus you've added commercials to the experienc...,None
2,570301083672813571,Virgin America,neutral,i didn't today... must mean i need to take ano...,None
3,570301031407624196,Virgin America,negative,"it's really aggressive to blast obnoxious ""ent...",Bad Flight
4,570300817074462722,Virgin America,negative,and it's a really big bad thing about it,Can't Tell


Loaded 14,640 tweets


## 2 Train / Validation Split

Create a **90 % / 10 % stratified split** so that class ratios (`negative`, `neutral`, `positive`) stay identical in training and validation sets.

In [ ]:
train_df, val_df = train_test_split(
    df,
    test_size   = 0.10,
    stratify    = df["airline_sentiment"],
    random_state= SEED,
)

print(f"Train rows: {len(train_df):,}  │ Val rows: {len(val_df):,}")

Train rows: 13,176  │ Val rows: 1,464


## 3 Tokenisation → HF Datasets

1. Build a label ↔ ID mapping.  
2. Use DistilBERT’s tokenizer to turn each tweet into `input_ids` and `attention_mask`.  
3. Convert pandas DataFrames into **`datasets.Dataset`** objects for high‑speed, on‑disk caching.  
4. Remove raw text columns so the dataset now holds **tensors only** (`input_ids`, `attention_mask`, `labels`).

In [5]:
LABELS   = ["negative", "neutral", "positive"]
label2id = {lab: i for i, lab in enumerate(LABELS)}
id2label = {i: lab for lab, i in label2id.items()}

tok = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def encode(batch):
    enc = tok(
        batch["clean_text"],
        truncation=True,
        max_length=128,
    )
    enc["labels"] = [label2id[x] for x in batch["airline_sentiment"]]
    return enc

cols = ["clean_text", "airline_sentiment"]

train_ds = Dataset.from_pandas(train_df[cols]).map(encode, batched=True,
                                                   remove_columns=cols)
val_ds   = Dataset.from_pandas(val_df[cols]).map(encode, batched=True,
                                                 remove_columns=cols)

print(train_ds)
print(val_ds)

Map: 100%|██████████| 1464/1464 [00:00<00:00, 29385.40 examples/s]

Dataset({
    features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 13176
})
Dataset({
    features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1464
})


## 4 Model Instantiation

Load DistilBERT with a **new classification head** sized for 3 labels.  
Hugging Face warns that the classification weights are randomly initialised—exactly what we want before fine‑tuning.

In [6]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels = len(LABELS),
    id2label   = id2label,
    label2id   = label2id,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 5 Training Arguments

Define *how* we train:

* 2 epochs, batch‑size 16, learning‑rate 2 × 10⁻⁵  
* Evaluate and save a checkpoint **once per epoch**  
* Basic weight‑decay and logging cadence

> **Note** Older versions of Transformers expect `eval_strategy`  
> whereas ≥ 3.4 use `evaluation_strategy`.

In [7]:
EPOCHS      = 2
BATCH_SIZE  = 16
LEARNING_RATE = 2e-5

train_args = TrainingArguments(
    output_dir              = MODEL_DIR,
    eval_strategy     = "epoch",
    save_strategy           = "epoch",
    learning_rate           = LEARNING_RATE,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size  = BATCH_SIZE,
    num_train_epochs        = EPOCHS,
    weight_decay            = 0.01,
    logging_steps           = 50,
    seed                    = SEED,
    report_to               = "none",
)


## 6 Trainer + Fine‑Tune

Glue everything together:

1. **DataCollatorWithPadding** dynamically pads each batch.  
2. **compute_metrics** returns accuracy and macro‑F1 after every validation pass.  
3. **Trainer.train()** runs the full training loop and prints a neat progress bar plus validation scores.

In [8]:
data_collator = DataCollatorWithPadding(tokenizer=tok, return_tensors="pt")

metric_acc = load_metric("accuracy")
metric_f1  = load_metric("f1")

def compute_metrics(eval_pred):
    preds = eval_pred.predictions.argmax(-1)
    refs  = eval_pred.label_ids
    return {
        "accuracy": metric_acc.compute(predictions=preds, references=refs)["accuracy"],
        "f1": metric_f1.compute(predictions=preds, references=refs, average="macro")["f1"],
    }

trainer = Trainer(
    model           = model,
    args            = train_args,
    train_dataset   = train_ds,
    eval_dataset    = val_ds,
    data_collator   = data_collator,
    compute_metrics = compute_metrics,
)

trainer.train()

c:\Users\justi\Anaconda3\envs\twitter-sentiment-env\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.416100,0.444006,0.835383,0.782617
2,0.333600,0.450154,0.838798,0.791774


c:\Users\justi\Anaconda3\envs\twitter-sentiment-env\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=1648, training_loss=0.41087919589385247, metrics={'train_runtime': 1711.3091, 'train_samples_per_second': 15.399, 'train_steps_per_second': 0.963, 'total_flos': 256589787423456.0, 'train_loss': 0.41087919589385247, 'epoch': 2.0})

## 7 Results Snapshot

After 2 epochs you should see something like:

| Epoch | Train Loss | Validation Loss | Accuracy | F1 |
|-------|-----------:|---------------:|---------:|---:|
| 1 | 0.41 | 0.33 | 0.78 | 0.78 |
| 2 | 0.33 | 0.26 | 0.84 | 0.84 |

*Numbers will vary slightly depending on seed and hardware.*


In [9]:
val_metrics = trainer.evaluate()
val_metrics

c:\Users\justi\Anaconda3\envs\twitter-sentiment-env\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.4501541554927826,
 'eval_accuracy': 0.8387978142076503,
 'eval_f1': 0.7917735076611953,
 'eval_runtime': 18.4713,
 'eval_samples_per_second': 79.258,
 'eval_steps_per_second': 4.981,
 'epoch': 2.0}

## 8 Save Artifacts & Export

Persist everything required for later inference or sharing:

* **Fine‑tuned model weights** (`models/distilbert_twitter/final/`)  
* **Tokenizer vocab & config** (`models/distilbert_twitter/tokenizer/`)  
* **Validation metrics** as a tiny CSV for easy comparison

In [ ]:
trainer.save_model(MODEL_DIR / "final")
tok.save_pretrained(MODEL_DIR / "tokenizer")
pd.DataFrame([val_metrics]).to_csv(MODEL_DIR / "metrics.csv", index=False)